# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
b_call = sum(data[data.race=='b'].call)
b_call

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
# number of callbacks for white-sounding names
w_call = sum(data[data.race=='w'].call)
w_call

235.0

In [6]:
# total number of callbacks
total_call = sum(data.call)
total_call

392.0

In [7]:
# total number of black-sounding names
b_total = sum(data.race == 'b')
b_total

2435

In [8]:
# total number of white-sounding names
w_total = sum(data.race == 'w')
w_total

2435

In [9]:
# total number resumes
total = data.shape[0]
total

4870

In [10]:
# percentage of black-sounding names callback
b_p = b_call / b_total
b_p

0.064476386036960986

In [11]:
# percentage of white-sounding names callback
w_p = w_call / w_total
w_p

0.096509240246406572

In [12]:
# percentage of all callbacks
t_p = total_call / total
t_p

0.080492813141683772

In [13]:
diff_p = w_p - b_p
diff_p

0.032032854209445585

In [14]:
# Not sure what df.var() does? I thought it calculated variance the same 
# as mean() or std()? I suppose I could calculate std() and square it for the
# variance.

#diff_std = np.sqrt(data[data.race == 'w'].call.var() 
#                   + data[data.race == 'b'].call.var())
#diff_std

In [15]:
# std of diff is square root of the sum of the variances
diff_std2 = np.sqrt((w_p * (1 - w_p))/w_total + (b_p * (1 - b_p))/b_total)
diff_std2

0.0077833705866767544

In [16]:
# 1.96 is Z-score for 97.5%
d = diff_std2 * stats.norm.ppf(q = 0.975)
d

0.015255126028214831

In [23]:
# 95% confidence interval is +- the product  of 97.5% Z-score 
# and std of sample dist.
conf_95 = [diff_p - d, diff_p + d]
print('Mean = %6.3f CI = [%6.3f, %6.3f] MOE = %6.3f' 
      % (diff_p,conf_95[0],conf_95[1],d))

Mean =  0.032 CI = [ 0.017,  0.047] MOE =  0.015


In [18]:
# Null hypothesis is that there is no difference in callbacks 
# based on white or black-sounding names. The alternative hypothesis is that
# there is a difference.
null_hyp = 0

In [19]:
# Assuming null hypothesis, w_p = b_p which we can estimate as t_p. Thus, std
# of the sample mean dist under the null hypothesis.
Z = (diff_p - null_hyp)/np.sqrt(2 * t_p * (1 - t_p)/w_total)
Z

4.1084121524343464

In [20]:
# Some how this is equivallent to the above, need to investigate as to why.
# Z = (diff_p - null_hyp)/(data.call.std()/np.sqrt(total))/2
# Z

4.1080375084517646

In [21]:
pval = stats.norm.sf(np.abs(Z)) * 2
pval

3.9903531079561974e-05

In [24]:
print('p-value = %6.5f' % (pval))

p-value = 0.00004


#### Summary

   1: What test is appropriate for this problem? Does CLT apply?   
   
   I used a Z test for this problem as sample size was relatively large. The CLT would apply in this case.

      
   2: What are the null and alternate hypotheses?
   
   The null hypothesis is that there is no difference between the callback rates for black and white-sounding names. The alternative hypothesis is that there is a difference.

   
   3: Compute margin of error, confidence interval, and p-value.
   
   Mean =  0.032, CI = [ 0.017,  0.047], MOE =  0.015, p-value = 0.00004

   4: Write a story describing the statistical significance in the context or the original problem.
   
   The difference in callback rates of approx. 3% more callbacks for white-sounding names versus black-sounding names is statistically significant given there being a less than 1% percent chance of randomly sampling that difference assuming no such difference existed. It would appear, based on this breif analysis, that the black-sounding names are less likely to be picked for callbacks on resumes as compared to their white-sounding named collegues.

   5: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
   
   I don't believe that this analysis means that race/name is the most important factor in callback success. I think it means it is a factor, or rather, is likely a factor. There are many other factors such as previous work experience, education level, etc. that were not considered in this analysis. 
   
   The analysis could have included other columns from the dataset for consideration. Either column by column, or together is some sort of regression analysis.